OpenCV 사용


일단, 비디오를 불러와서 프레임단위로 추출 + 저장이 1차 목적

<참고>

https://docs.opencv.org/2.4/modules/highgui/doc/reading_and_writing_images_and_video.html 




* videoCapture 인터페이스를 이용한 베이직 코드


opencv_source_code/samples/python2/video.py




* basic 비디오 처리 샘플코드


opencv_source_code/samples/python2/video_dmtx.py




* 멀티쓰레드 비디오 처리 샘플코드


opencv_source_code/samples/python2/video_threaded.py

대기시간
k = cv2.waitKey(0)
if k == 27: #esc key
cv2.destroyAllWindows()
break

In [2]:
import os

def createfolder(folder):
    try:
        if not os.path.exists(folder):
            os.makedirs(folder)
    except OSError:
        print('Error: creating directory' + folder)


In [3]:
import PIL
from PIL import Image

def load_image_file(file, mode='RGB'):
    im = PIL.Image.open(file)
    if mode:
        im = im.convert(mode)
    return np.array(im)

## path 경로 설정 해야해 미현아!!! 


import cv2

path = '/Users/hyeonjeong_baek/CyberProjects/Deep-t-ADIS/W8_ExtractFrame/sample3_frame'
video = cv2.VideoCapture('sample3.mp4')
count = 0

videoframe = video.get(cv2.CAP_PROP_POS_FRAMES)
videocount = video.get(cv2.CAP_PROP_FRAME_COUNT)

createfolder(path)

while video.isOpened():
    #ret 은 프레임이 존재하지 않을때 T/F 반환
    #frame 은 프레임 반환
    if(videoframe == videocount):
        video.open('sample3.mp4')
    
    ret, frame = video.read()
    if ret == False:
        print("Frame doesn't Exist")
        break
    
    try:
        cv2.imshow('sample3', frame)        
        cv2.imwrite('sample3_frame/frame %d.jpg' %count, frame)
        #print('Saved frame%d.jpg' %count)
        count += 1
        
    except :
        pass

video.release()
cv2.destroyAllWindows()



**face_recognition 의 load_image_file 함수가 안먹어서 api 코드보고 바꿈**

import os

directory = os.walk(path).__next__()[2]
print(len(directory))

## 샘플(연희얼굴 - frame 두 개만|)

import face_recognition

images = []
for i in range(2):
    image = load_image_file("sample3_frame/frame "+str(i)+".jpg")
    images.append(image)

face_locations = []
for img in images:
    print(img)
    face_location = face_recognition.face_locations(img)
    #face_locations = face_recognition.face_locations(image)
    face_locations.append(face_location)

    
print(face_locations)
print("I found {} face(s) in this photograph.".format(len(face_locations)))

for face_location in face_locations:
    print(face_location)
    #print("image", image)
    # Print the location of each face in this image
    top, right, bottom, left = face_location[0]
    print("A face is located at pixel location Top: {}, Left: {}, Bottom: {}, Right: {}".format(top, left, bottom, right))

    # You can access the actual face itself like this:
    face_image = image[top:bottom, left:right]
    pil_image = Image.fromarray(face_image)
    pil_image.show()

## face_recognition 라이브러리를 이용해서 face frame 추출

**지금은 video -> frame 이미지 저장 -> 이미지 불러와서 얼굴만 따오기** 

이거를 아예 video -> 얼굴 프레임 저장으로 간추릴 예정임

import numpy as np
import face_recognition


#image = face_recognition.load_image_file("/sample2_frame/frame 0.jpg")

images = []
#for i in range(len(os.walk(path).__next__()[2])):
for i in range(100):
    image = load_image_file("sample3_frame/frame "+str(i)+".jpg")
    images.append(image)
    

face_locations = []
for img in images:
    face_location = face_recognition.face_locations(img, model="cnn")
    
    '''여기 model="cnn" 하면 원래 된다는데 내 컴퓨터로는 안됨 ㅜㅜㅜㅜ'''
    
    
    #face_locations = face_recognition.face_locations(image)
    #print("I found {} face(s) in this photograph.".format(len(face_locations)))
    face_locations.append(face_location)
    
print(face_locations)

for face_location in face_locations:

    # Print the location of each face in this image
    top, right, bottom, left = face_location[0]
    print("A face is located at pixel location Top: {}, Left: {}, Bottom: {}, Right: {}".format(top, left, bottom, right))

    # You can access the actual face itself like this:
    face_image = image[top:bottom, left:right]
    pil_image = Image.fromarray(face_image)
    pil_image.show()

https://face-recognition.readthedocs.io/en/latest/_modules/face_recognition/api.html#load_image_file 

# step 1

## mp4 파일 -> face frame 으로 바로 저장

In [4]:
import cv2
import numpy as np
import face_recognition

videoname = 'sample3'
path = '/Users/hyeonjeong_baek/CyberProjects/Deep-t-ADIS/W8_ExtractFrame/'+videoname+'_frame'
video = cv2.VideoCapture(videoname+'.mp4')
count = 0
faces = []

videoframe = video.get(cv2.CAP_PROP_POS_FRAMES)
videocount = video.get(cv2.CAP_PROP_FRAME_COUNT)

createfolder(path)

while video.isOpened():
    #ret 은 프레임이 존재하지 않을때 T/F 반환
    #frame 은 프레임 반환
    if(videoframe == videocount):
        video.open(videoname+'.mp4')
    
    ret, frame = video.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    if ret == False:
        print("Frame doesn't Exist") 
        break
    
    try:
        #얼굴 detection 한 프레임으로 
        #face_location 은 얼굴 위치(행렬에서)
        face_location = face_recognition.face_locations(frame)  #여기에 model="cnn" 넣으면 cnn모델로..
        top, right, bottom, left = face_location[0]
        print("A face is located at pixel location Top: {}, Left: {}, Bottom: {}, Right: {}".format(top, left, bottom, right))

        #face_location 찾아서 파일로 저장
        face_image = Image.fromarray(frame[top:bottom, left:right])
        #Image.fromarray(face_image).show()
        face_image.save(videoname+'_frame/frame %d.jpg' %count)
        #print('Saved frame%d.jpg' %count)
        count += 1
        
    except :
        pass

print(faces)
video.release()
cv2.destroyAllWindows()



A face is located at pixel location Top: 384, Left: 776, Bottom: 705, Right: 1097
A face is located at pixel location Top: 384, Left: 776, Bottom: 705, Right: 1097
A face is located at pixel location Top: 384, Left: 776, Bottom: 705, Right: 1097
A face is located at pixel location Top: 384, Left: 776, Bottom: 705, Right: 1097
A face is located at pixel location Top: 384, Left: 776, Bottom: 705, Right: 1097
A face is located at pixel location Top: 384, Left: 776, Bottom: 705, Right: 1097
A face is located at pixel location Top: 384, Left: 776, Bottom: 705, Right: 1097
A face is located at pixel location Top: 384, Left: 776, Bottom: 705, Right: 1097
A face is located at pixel location Top: 384, Left: 776, Bottom: 705, Right: 1097
A face is located at pixel location Top: 384, Left: 776, Bottom: 705, Right: 1097
A face is located at pixel location Top: 333, Left: 718, Bottom: 718, Right: 1103
A face is located at pixel location Top: 333, Left: 718, Bottom: 718, Right: 1103
A face is locate

A face is located at pixel location Top: 420, Left: 633, Bottom: 741, Right: 954
A face is located at pixel location Top: 420, Left: 633, Bottom: 741, Right: 954
A face is located at pixel location Top: 376, Left: 589, Bottom: 761, Right: 974
A face is located at pixel location Top: 376, Left: 589, Bottom: 761, Right: 974
A face is located at pixel location Top: 420, Left: 598, Bottom: 741, Right: 919
A face is located at pixel location Top: 376, Left: 546, Bottom: 761, Right: 932
A face is located at pixel location Top: 333, Left: 546, Bottom: 718, Right: 932
A face is located at pixel location Top: 297, Left: 502, Bottom: 759, Right: 964
A face is located at pixel location Top: 333, Left: 504, Bottom: 718, Right: 889
A face is located at pixel location Top: 246, Left: 451, Bottom: 708, Right: 913
A face is located at pixel location Top: 246, Left: 451, Bottom: 708, Right: 913
A face is located at pixel location Top: 194, Left: 399, Bottom: 657, Right: 861
A face is located at pixel l

A face is located at pixel location Top: 350, Left: 1033, Bottom: 617, Right: 1300
A face is located at pixel location Top: 350, Left: 1062, Bottom: 617, Right: 1330
A face is located at pixel location Top: 350, Left: 1062, Bottom: 617, Right: 1330
A face is located at pixel location Top: 379, Left: 1062, Bottom: 647, Right: 1330
A face is located at pixel location Top: 350, Left: 914, Bottom: 617, Right: 1181


error: OpenCV(4.2.0) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


# step 2


## 얼굴 feature 추출 -> 2D facial landmark 표시하기

https://github.com/1adrianb/face-alignment

In [9]:
import face_alignment
from skimage import io
import os

framepath = videoname+'_frame/frame '

fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._3D, flip_input=False)

for i in range(len(os.walk(path).__next__()[2])):
    input = io.imread(framepath+str(i)+'.jpg')
    preds = fa.get_landmarks(input)

AssertionError: Torch not compiled with CUDA enabled

conda 로 할 때 이것저것 



conda install cmake

conda install -c menpo dlib

conda install face_recognition


conda install pandas, pytorch, torchvision 등등